# Create and optimize a quadratic optimization problem in CPLEX solver

In [3]:
conda create - name MIE1622
conda activate MIE1622

conda install -c conda- forge cvxpy

SyntaxError: invalid syntax (<ipython-input-3-caae06806ea6>, line 1)

### Quadratic optimization problem

\begin{equation}
\begin{array}{rl}
\displaystyle \mbox{max}_{x} & x_1 + 2 x_2 + 3 x_3 - 0.5 ( 33 x_1^2 + \\
& 22 x_2^2 + 11 x_3^2 -  12 x_1 x_2 - 23 x_2 x_3 ) \\
{\rm s.t.} & - x_1 + x_2 + x_3 \leq 20 \\
& x_1 - 3 x_2 + x_3 \leq 30 \\
& 0 \leq x_1 \leq 40 \\
& x_2 \geq 0,~ x_3 \geq 0
\end{array}
\end{equation}

### Import libraries

Install CPLEX solver. To setup CPLEX solver for Python:
* Use the script `setup.py` located in the directory `yourCPLEXhome/python`
* Execute the following command from the command line `python yourCPLEXhome/python/setup.py install` or `python yourCPLEXhome/python/setup.py install --home yourPythonPackageshome`
* Set the environment variable `PYTHONPATH` to `yourCPLEXhome/python/VERSION/PLATFORM`

In [1]:
from __future__ import print_function
import cplex

### Define CPLEX model

Initialize CPLEX object

In [2]:
cpx = cplex.Cplex()
cpx.objective.set_sense(cpx.objective.sense.maximize)

Define linear part of objective function and bounds on variables

In [3]:
c = [1.0, 2.0, 3.0]
ub = [40.0, cplex.infinity, cplex.infinity]

Define constraints `cols` and add right-hand-side (`rhs`) of constraints

In [4]:
cols = [[[0, 1], [-1.0, 1.0]],
        [[0, 1], [1.0, -3.0]],
        [[0, 1], [1.0, 1.0]]]
cpx.linear_constraints.add(rhs=[20.0, 30.0], senses="LL")

range(0, 2)

Add linear part of objective function, bounds on variables, and linear constraints to CPLEX model

In [5]:
cpx.variables.add(obj=c, ub=ub, columns=cols,
                  names=["one", "two", "three"])

range(0, 3)

Add quadratic part of objective function to CPLEX model

In [6]:
qmat = [[[0, 1], [-33.0, 6.0]],
        [[0, 1, 2], [6.0, -22.0, 11.5]],
        [[1, 2], [11.5, -11.0]]]
cpx.objective.set_quadratic(qmat)

Set CPLEX parameters

In [7]:
alg = cpx.parameters.lpmethod.values
print("Setting solution algorithm = ", alg.concurrent)
cpx.parameters.qpmethod.set(alg.concurrent)

Setting solution algorithm =  6


### Solve CPLEX model

Optimize the problem

In [8]:
cpx.solve()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_QPMethod                                6
Parallel mode: deterministic, using up to 8 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 6 threads...
 * Starting primal Simplex on 1 thread...
Number of nonzeros in lower triangle of Q = 2
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.02 sec. (0.00 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 3
  Integer space required    = 3
  Total non-zeros in factor = 6
  Total FP ops to factor    = 14
Tried aggregator 1 time.
No QP presolve or aggregator reductions.
Presolve time = 0.03 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1    Phase I obj   =             0.015152

Dual simplex solved model.



Display the solution

In [9]:
# solution.get_status() returns an integer code
print("Solution status = ", cpx.solution.get_status(), ":", end=' ')
# the following line prints the corresponding string
print(cpx.solution.status[cpx.solution.get_status()])
print("Solution value  = ", cpx.solution.get_objective_value())

Solution status =  1 : optimal
Solution value  =  2.015616523289157


In [10]:
numcols = cpx.variables.get_num()
for j in range(numcols):
    print("Column ", j, ":  ", end=' ')
    print("Value = %10f " % cpx.solution.get_values(j), end=' ')
    print("Reduced Cost = %10f" % cpx.solution.get_reduced_costs(j))

numrows = cpx.linear_constraints.get_num()
for i in range(numrows):
    print("Row ", i, ":  ", end=' ')
    print("Slack = %10f " % cpx.solution.get_linear_slacks(i), end=' ')
    print("Pi = %10f" % cpx.solution.get_dual_values(i))

Column  0 :   Value =   0.139115  Reduced Cost =  -0.000000
Column  1 :   Value =   0.598465  Reduced Cost =  -0.000000
Column  2 :   Value =   0.898396  Reduced Cost =  -0.000000
Row  0 :   Slack =  18.642254  Pi =  -0.000000
Row  1 :   Slack =  30.757886  Pi =  -0.000000


### Solve CPLEX model using CVXPY modeling environment

CVXPY is a Python-embedded modeling language for convex optimization problems. It automatically transforms the problem into standard form, calls a solver, and unpacks the results.

CVXPY can be installed with `conda install -c conda-forge cvxpy` or `pip install cvxpy`.

In [11]:
#!pip install cvxpy

In [12]:
import cvxpy as cp
import numpy as np

In [13]:
Q = np.array([[-33.0, 6.0, 0.0], [6.0, -22.0, 11.5], [0., 11.5, -11.0]])
c = np.array([1., 2., 3.])
A = np.array([[-1., 1., 1.], [1., -3., 1.]])
b = np.array([20., 30.])
ub = np.array([40., np.inf, np.inf])
x = cp.Variable(3)
prob = cp.Problem(cp.Maximize((1/2)*cp.quad_form(x, Q) + c.T@x),
                 [A@x <= b,
                  x >= 0, x <= ub])
#prob = cp.Problem(cp.Maximize((1/2)*cp.quad_form(x, Q) + c.T@x),
#                 [A@x <= b,
#                  x >= 0, x[0] <= 40.])
prob.solve(solver=cp.CPLEX, verbose=True,cplex_params={"qpmethod": 6})

# Print result.
print("\nSolution status: ", prob.status)
print("Solution optimal value: ", prob.value)
print("Solution x: ")
print(x.value)
print("A dual solution corresponding to the inequality constraints: ")
print(prob.constraints[0].dual_value)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_QPMethod                                6
Parallel mode: deterministic, using up to 8 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 6 threads...
 * Starting primal Simplex on 1 thread...
Number of nonzeros in lower triangle of Q = 2
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.01 sec. (0.00 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 3
  Integer space required    = 3
  Total non-zeros in factor = 6
  Total FP ops to factor    = 14
Tried aggregator 1 time.
QP Presolve eliminated 8 rows and 0 columns.
Number of nonzeros in lower triangle of Q = 2
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.02 sec. (0.00 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 3
  Integer space required    = 3
  Total non-zero